# Structured Configs

Following: https://omegaconf.readthedocs.io/en/latest/structured_config.html

> Structured configs are used to create OmegaConf configuration object **with runtime type safety**.
>
> In addition, they can be used *with tools like mypy or your IDE* for **static type checking**.


Two types of structures classes are supported: `dataclasses` and `attr` classes.
* `dataclass`es are standard as of Python 3.7 or newer and are available in Python 3.6 via the `dataclasses` pip package.
    * NOTE: official Python `dataclass`es: https://docs.python.org/3.7/library/dataclasses.html
* `attrs` Offset slightly cleaner syntax in some cases but depends on the attrs pip package.
    * Uses a *library*: https://github.com/python-attrs/attrs

**NOTE: It can use *EITHER* of the above, it doesn't use BOTH**

> This documentation will use `dataclass`es, but you can use the annotation `@attr.s(auto_attribs=True)`
> from `attrs` instead of `@dataclass`.

Basic usage involves passing in a *structured config class or instance* to `OmegaConf.structured()`, which will return an OmegaConf config that matches the values and types specified in the input.

At runtime, OmegaConf will validate modifications to the created config object against the schema specified in the input class.


Currently, type hints supported in OmegaConf’s structured configs include:
* primitive types (`int`, `float`, `bool`, `str`, `bytes`, `Path`) and enum types (user-defined subclasses of `enum.Enum`). See the **Simple types** section below.
* unions of primitive/enum types, e.g. `Union[float, bool, MyEnum]`. See **Unions** below.
* structured config fields (i.e. `MyConfig.x` can have type hint `MySubConfig`). See the **Nesting structured configs** section below.
* `dict` and `list` types: `typing.Dict[K, V]` or `typing.List[V]`, where `K` is primitive or enum, and where `V` is any of the above (including nested dicts or lists, e.g. `Dict[str, List[int]]`). See the **Lists and Dictionaries sections** below.
* optional types (any of the above can be wrapped in a `typing.Optional[...]` annotation). See **Other special features** below.


## Simple types


Simple types include:
* `int`: numeric integers
* `float`: numeric floating point values
* `bool`: boolean values (`True`, `False`, On, Off etc.)
* `str`: any string
* `bytes`: an immutable sequence of numbers in `[0, 255]`
* `pathlib.Path`: filesystem paths as represented by python’s standard library `pathlib`
* `Enums`: User defined enums

The following class defines fields with all simple types:

In [1]:
from enum import Enum
from dataclasses import dataclass
import pathlib


class Height(Enum):
    SHORT = 0
    TALL = 1


@dataclass
class SimpleTypes:
    num: int = 10
    pi: float = 3.1415
    is_awesome: bool = True
    height: Height = Height.SHORT
    description: str = "text"
    data: bytes = b"bin_data"
    path: pathlib.Path = pathlib.Path("hello.txt")

You can create a config based on the *SimpleTypes class* itself or *an instance of it*.

Those would be equivalent by default, but the *Object* variant allows you to *set the values of specific fields during construction*.

In [2]:
from omegaconf import OmegaConf

conf1 = OmegaConf.structured(SimpleTypes)
conf2 = OmegaConf.structured(SimpleTypes())

# The two configs are identical in this case
assert conf1 == conf2

# But the second form allow for easy customization of the values:
conf3 = OmegaConf.structured(SimpleTypes(num=20, height=Height.TALL))

print(OmegaConf.to_yaml(conf3))

num: 20
pi: 3.1415
is_awesome: true
height: TALL
description: text
data: !!binary |
  YmluX2RhdGE=
path: !!python/object/apply:pathlib.PosixPath
- hello.txt



The resulting object is a regular OmegaConf `DictConfig`,
> **except** that it will utilize the type information in the input class/object and will validate the data at runtime.

The resulting object will also **rejects attempts to access or set fields that are not already defined** (similarly to configs with their to *Struct* flag set, but not recursive).

In [10]:
conf = OmegaConf.structured(SimpleTypes)

print(conf)
display(type(conf))

try:
    conf.does_not_exist
except AttributeError as e:
    print(e)

{'num': 10, 'pi': 3.1415, 'is_awesome': True, 'height': <Height.SHORT: 0>, 'description': 'text', 'data': b'bin_data', 'path': PosixPath('hello.txt')}


omegaconf.dictconfig.DictConfig

Key 'does_not_exist' not in 'SimpleTypes'
    full_key: does_not_exist
    object_type=SimpleTypes


## Static type checker support

Python type annotation can be used by static type checkers like *Mypy/Pyre* or by IDEs like *PyCharm*.

In [11]:
from omegaconf import ValidationError


conf: SimpleTypes = OmegaConf.structured(SimpleTypes)

# Passes static type checking
conf.description = "text"

# Fails static type checking (but will also raise a Validation error)
try:
    conf.num = "foo"  # pyright: ignore
except ValidationError as e:
    print(e)

Value 'foo' of type 'str' could not be converted to Integer
    full_key: num
    object_type=SimpleTypes


This is **duck-typing**; the actual object type of conf is `DictConfig`.

You can access the *underlying type* using `OmegaConf.get_type()`:

In [12]:
type(conf).__name__

'DictConfig'

In [16]:
# NOTE: This is important to understand! Compare with the above line.
OmegaConf.get_type(conf).__name__  # pyright: ignore

'SimpleTypes'

## Runtime type validation and conversion

OmegaConf supports merging configs together, as well as overriding from the command line.

This means *some mistakes can not be identified by static type checkers*, and **runtime validation is required**.

In [17]:
# This is okay, the string "100" can be converted to an int
# Note that static type checkers will not like it and you should
# avoid such explicit mistyped assignments.

conf.num = "100"  # pyright: ignore
assert conf.num == 100
# NOTE: ^ Note that "coercion" exists similar to pydantic.

try:
    # This will fail at runtime because num is an int
    # and foo cannot be converted to an int
    # Note that the static type checker can't help here.
    conf.merge_with_dotlist(["num=foo"])  # pyright: ignore

except ValidationError as e:
    print(e)

Value 'foo' of type 'str' could not be converted to Integer
    full_key: num
    object_type=SimpleTypes


Runtime validation and conversion works for all supported types, including `Enum`s:

In [18]:
conf.height = Height.TALL

assert conf.height == Height.TALL

# The name of Height.TALL is TALL
conf.height = "TALL"  # pyright: ignore
assert conf.height == Height.TALL

# This works too
conf.height = "Height.TALL"  # pyright: ignore
assert conf.height == Height.TALL

# The ordinal of Height.TALL is 1
conf.height = 1  # pyright: ignore
assert conf.height == Height.TALL

## Nesting structured configs

Structured configs can be nested.

In [19]:
from omegaconf import MISSING
from dataclasses import field  # NOTE!


@dataclass
class User:
    # A simple user class with two missing fields
    name: str = MISSING
    height: Height = MISSING


@dataclass
class DuperUser(User):
    duper: bool = True


# Group class contains two instances of User.
@dataclass
class Group:
    name: str = MISSING

    # data classes can be nested
    admin: User = field(
        default_factory=User
    )  # NOTE: field > default factory pattern! (Python dataclasses thing).

    # You can also specify different defaults for nested classes
    manager: User = field(
        default_factory=lambda: User(name="manager", height=Height.TALL)
    )
    # NOTE: ^ field > default factory pattern! (Python dataclasses thing).


conf: Group = OmegaConf.structured(Group)
print(OmegaConf.to_yaml(conf))

name: ???
admin:
  name: ???
  height: ???
manager:
  name: manager
  height: TALL



OmegaConf will validate that assignment of nested objects is of the correct type:

In [20]:
try:
    conf.manager = 10  # pyright: ignore
except ValidationError as e:
    print(e)

Invalid type assigned: int is not a subclass of User. value: 10
    full_key: manager
    object_type=Group


You can assign subclasses:

In [21]:
conf.manager = DuperUser()

assert conf.manager.duper == True

## Lists

Structured Config fields annotated with `typing.List` or `typing.Tuple` can hold any type supported by OmegaConf
(`int`, `float`, `bool`, `str`, `bytes`, `pathlib.Path`, `Enum` or *Structured configs*).

In [22]:
from dataclasses import dataclass, field
from typing import List, Tuple  # NOTE.


@dataclass
class User:
    name: str = MISSING


@dataclass
class ListsExample:
    # Typed list can hold Any, int, float, bool, str,
    # bytes, pathlib.Path and Enums as well as arbitrary Structured configs.
    ints: List[int] = field(default_factory=lambda: [10, 20, 30])
    bools: Tuple[bool, bool] = field(default_factory=lambda: (True, False))
    users: List[User] = field(default_factory=lambda: [User(name="omry")])

OmegaConf verifies at runtime that your Lists contains only values of the correct type.

In the example below, the OmegaConf object conf (which is actually an instance of DictConfig) is duck-typed as `ListExample`.

In [23]:
conf: ListsExample = OmegaConf.structured(ListsExample)

# Okay, 10 is an int
conf.ints.append(10)

# Okay, "20" can be converted to an int
conf.ints.append("20")  # pyright: ignore

conf.bools.append(True)  # pyright: ignore
conf.users.append(User(name="Joe"))

# Not okay, 10 cannot be converted to a User
try:
    conf.users.append(10)  # pyright: ignore
except ValidationError as e:
    print(e)

Invalid type assigned: int is not a subclass of User. value: 10
    full_key: users[2]
    reference_type=List[User]
    object_type=list


## Dictionaries

Dictionaries are supported via annotation of structured config fields with `typing.Dict`.

*Keys* must be typed as one of `str`, `int`, `Enum`, `float`, `bytes`, or `bool`.

*Values* can be any of the types supported by OmegaConf (`Any`, `int`, `float`, `bool`, `bytes`, `pathlib.Path`, `str` and `Enum` as well as arbitrary *Structured configs*)

In [25]:
from dataclasses import dataclass, field
from typing import Dict


@dataclass
class DictExample:
    ints: Dict[str, int] = field(default_factory=lambda: {"a": 10, "b": 20, "c": 30})
    bools: Dict[str, bool] = field(default_factory=lambda: {"Uno": True, "Zoro": False})
    users: Dict[str, User] = field(default_factory=lambda: {"omry": User(name="omry")})

Like with Lists, the types of values contained in Dicts are verified at runtime.

In [26]:
conf: DictExample = OmegaConf.structured(DictExample)

# Okay, correct type is assigned
conf.ints["d"] = 10

conf.bools["Dos"] = True
conf.users["James"] = User(name="Bond")

# Not okay, 10 cannot be assigned to a User
try:
    conf.users["Joe"] = 10  # pyright: ignore
except ValidationError as e:
    print(e)

Invalid type assigned: int is not a subclass of User. value: 10
    full_key: Joe
    object_type=None


## Nested dict and list annotations

Dict and List annotations can be nested flexibly:

In [27]:
@dataclass
class NestedContainers:
    dict_of_dict: Dict[str, Dict[str, int]]
    list_of_list: List[List[int]] = field(default_factory=lambda: [[123]])
    dict_of_list: Dict[str, List[int]] = MISSING
    list_of_dict: List[Dict[str, int]] = MISSING


cfg = OmegaConf.structured(NestedContainers(dict_of_dict={"foo": {"bar": 123}}))

print(OmegaConf.to_yaml(cfg))

dict_of_dict:
  foo:
    bar: 123
list_of_list:
- - 123
dict_of_list: ???
list_of_dict: ???



In [28]:
try:
    cfg.list_of_dict = [["whoops"]]  # not a list of dicts
except ValidationError as e:
    print(e)

Invalid type assigned: list is not a subclass of Dict[str, int]. value: ['whoops']
    full_key: 0
    object_type=None


## Unions

You can use `typing.Union` to annotate unions of **Simple types** (⚠️).

In [31]:
from typing import Union


@dataclass
class HasUnion:
    u: Union[float, bool] = 10.1


cfg = OmegaConf.structured(HasUnion)

assert cfg.u == 10.1
cfg.u = True  # ok

try:
    cfg.u = b"binary"  # bytes not compatible with union
except ValidationError as e:
    print(e)

try:
    OmegaConf.structured(HasUnion("abc"))  # pyright: ignore  # str not compatible
except ValidationError as e:
    print(e)

Value 'b'binary'' of type 'bytes' is incompatible with type hint 'Union[float, bool]'
    full_key: u
    object_type=HasUnion
Value 'abc' of type 'str' is incompatible with type hint 'Union[float, bool]'
    full_key: u
    object_type=None


If any argument of a `Union` type hint is `Optional`, the whole union is considered optional.

ℹ️ For example, OmegaConf treats all four of the following type hints as equivalent:
* `Optional[Union[int, str]]`
* `Union[Optional[int], str]`
* `Union[int, str, None]`
* `Union[int, str, type(None)]`

**Ordinarily**, assignment to a structured config field results in coercion of the assigned value to the field’s type.

For example, assigning an integer to a field typed as `str` results in the integer being converted to a string:

In [32]:
@dataclass
class HasStr:
    s: str


cfg = OmegaConf.structured(HasStr)
cfg.s = 10.1

assert cfg.s == "10.1"  # The assigned value has been converted to a string

> ⚠️ When dealing with `Union` types, **however**, *conversion is disabled so as to avoid ambiguity*.

Values assigned to a union-typed field of a structured config must precisely match one of the types in the `Union` annotation:

In [33]:
@dataclass
class StrOrInt:
    u: Union[str, float]  # NOTE.


cfg = OmegaConf.structured(StrOrInt)

cfg.u = 10.1
assert cfg.u == 10.1  # NOTE: The assigned value remains a `float`.

cfg.u = "10.1"
assert cfg.u == "10.1"  # NOTE: The assigned value remains a `str`.

try:
    cfg.u = 123  # Conversion from `int` to `float` does not occur.
except ValidationError as e:
    print(e)

Value '123' of type 'int' is incompatible with type hint 'Union[str, float]'
    full_key: u
    object_type=StrOrInt


## Other special features

OmegaConf supports field modifiers such as `MISSING` and `Optional`.

In [34]:
from typing import Optional
from omegaconf import MISSING


@dataclass
class Modifiers:
    num: int = 10
    optional_num: Optional[int] = 10  # NOTE.
    another_num: int = MISSING  # NOTE.
    optional_dict: Optional[Dict[str, int]] = None
    list_optional: List[Optional[int]] = field(
        default_factory=lambda: [10, MISSING, None]
    )


conf: Modifiers = OmegaConf.structured(Modifiers)

> Note for Python3.6 users: pickling structured configs with complex type annotations, such as dict-of-list or list-of-optional, is not supported.

### Mandatory missing values

Fields assigned the constant `MISSING` do not have a value and the value **must be set prior to accessing the field**.

Otherwise a `MissingMandatoryValue` exception is raised.

In [35]:
from omegaconf import MissingMandatoryValue

try:
    x = conf.another_num
except MissingMandatoryValue as e:
    print(e)

conf.another_num = 20
assert conf.another_num == 20

Missing mandatory value: another_num
    full_key: another_num
    object_type=Modifiers


### Optional fields

In [37]:
try:
    # regular fields cannot be assigned None
    conf.num = None  # pyright: ignore
except ValidationError as e:
    print(e)

conf.optional_num = None
assert conf.optional_num is None
assert conf.list_optional[2] is None

field 'num' is not Optional
    full_key: num
    object_type=Modifiers


### Interpolations

Variable interpolation works normally with Structured configs, **but static type checkers may object to you assigning a string to another type**.

To **work around** this, use the *special functions* `omegaconf.SI` and `omegaconf.II` described below.

In [38]:
from omegaconf import SI, II


@dataclass
class Interpolation:
    val: int = 100

    # This will work, but static type checkers will complain
    a: int = "${val}"  # NOTE: As you can see, pyright complains here.

    # This is equivalent to the above, but static type checkers
    # will not complain
    b: int = SI("${val}")  # NOTE: ... but doesn't complain here.

    # This is syntactic sugar; the input string is
    # wrapped with ${} automatically.
    c: int = II("val")


conf: Interpolation = OmegaConf.structured(Interpolation)

assert conf.a == 100
assert conf.b == 100
assert conf.c == 100

Interpolated values are validated, and converted when possible, to the annotated type when the interpolation is accessed, e.g:

In [41]:
from omegaconf import II


@dataclass
class Interpolation:
    str_key: str = "string"
    int_key: int = II("str_key")


cfg = OmegaConf.structured(Interpolation)

try:
    cfg.int_key  # fails due to type mismatch
except Exception as e:
    display(type(e))
    print(e)

omegaconf.errors.InterpolationValidationError

While dereferencing interpolation '${str_key}': Value 'string' of type 'str' could not be converted to Integer
    full_key: int_key
    object_type=Interpolation


In [42]:
cfg.str_key = "1234"  # string value

assert cfg.int_key == 1234  # automatically convert str to int

> ⚠️ Annoyance / missing functionality
>
> Note however that this validation step is currently skipped for container node interpolations:

In [44]:
@dataclass
class NotValidated:
    some_int: int = 0
    some_dict: Dict[str, str] = II("some_int")


cfg = OmegaConf.structured(NotValidated)

# NOTE:
assert cfg.some_dict == 0  # type mismatch, but no error

### Frozen classes

**Frozen** dataclasses and attr classes are supported via OmegaConf *Read-only flag*, which makes the entire config node and all of its child nodes read-only.

In [45]:
from dataclasses import dataclass, field
from typing import List
from omegaconf import ReadonlyConfigError


@dataclass(frozen=True)  # NOTE.
class FrozenClass:
    x: int = 10
    list: List = field(default_factory=lambda: [1, 2, 3])


conf = OmegaConf.structured(FrozenClass)

try:
    conf.x = 20
except ReadonlyConfigError as e:
    print(e)

Cannot change read-only config container
    full_key: x
    object_type=FrozenClass


In [46]:
# The read-only flag is recursive:

try:
    conf.list[0] = 20
except ReadonlyConfigError as e:
    print(e)

ListConfig is read-only
    full_key: list[0]
    reference_type=List[Any]
    object_type=list


## Merging with other configs

Once an OmegaConf object is created, it can be merged with others regardless of its source.

OmegaConf configs created from *Structured configs* contains type information that is enforced at runtime.

> This can be used to **validate config files based on a *schema* specified in a structured config class**.

`example.yaml` file:
```yaml
server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
  - user1
  - user2
```

A *Schema* for the above config can be defined like this.

In [47]:
@dataclass
class Server:
    port: int = MISSING


@dataclass
class Log:
    file: str = MISSING
    rotation: int = MISSING


@dataclass
class MyConfig:
    server: Server = field(
        default_factory=Server
    )  # NOTE the field > default_factory pattern.
    log: Log = field(default_factory=Log)
    users: List[int] = field(
        default_factory=list
    )  # NOTE: intentional error for the sake of the example.

I intentionally made an error in the type of the users list (`List[int]` should be `List[str]`).

This will cause a validation error when merging the config from the file with that from the scheme.

In [48]:
schema = OmegaConf.structured(MyConfig)

conf = OmegaConf.load("source/example.yaml")

try:
    OmegaConf.merge(schema, conf)
except ValidationError as e:
    print(e)

Value 'user1' of type 'str' could not be converted to Integer
    full_key: users[0]
    reference_type=List[int]
    object_type=list


## Using Metadata to Ignore Fields

OmegaConf inspects the metadata of `dataclass` / `attr` class fields, **ignoring any fields where `metadata["omegaconf_ignore"]`** is `True`.

When defining a `dataclass` or `attr` class, fields **can be given metadata** by passing the `metadata` keyword argument to the `dataclasses.field` function or the `attrs.field` function:

In [50]:
@dataclass
class HasIgnoreMetadata:
    normal_field: int = 1
    field_ignored: int = field(
        default=2, metadata={"omegaconf_ignore": True}
    )  # NOTE metadata.
    field_not_ignored: int = field(
        default=3, metadata={"omegaconf_ignore": False}
    )  # NOTE metadata.


cfg = OmegaConf.create(HasIgnoreMetadata)  # pyright: ignore

cfg

{'normal_field': 1, 'field_not_ignored': 3}

In the above example, `field_ignored` is ignored by OmegaConf.